# Generating COMBINE Archives from the SBML Test Suite

The [SBML test suite](http://sbml.org/Software/SBML_Test_Suite) is intended to test the accuracy SBML ODE simulators by providing a normative standard to compare simulation results to. The test suite predates SED-ML and specifies the simulation parameters in a custom format. However, recent versions include the SED-ML necessary to reproduce the simulations. Here, we provide an automated script to convert these test cases into COMBINE archives, which can be imported in Tellurium.

This notebook is adapted from Stanley Gu's original [here](http://nbviewer.jupyter.org/github/stanleygu/sbmltest2archive/blob/master/create_archives.ipynb).

In [ ]:
# should we attempt to download and convert the test cases?
# set to False to just run COMBINE archives
convert = True
# subset of the cases to run
cases = ['00001','00002','00003']

## Step 1: Change to the directory of this notebook

In [17]:
%cd ~/devel/src/tellurium-combine-archive-test-cases/sbml-test-suite

/Users/phantom/devel/src/tellurium-combine-archive-test-cases/sbml-test-suite


## Step 2: Download the SBML test cases

In [18]:
import urllib.request
import os.path

# url for the test case archive
url = 'http://sourceforge.net/projects/sbml/files/test-suite/3.1.1/cases-archive/sbml-test-cases-2014-10-22.zip'

test_cases_filename = 'sbml-test-cases.zip'

# download the test case archive
with urllib.request.urlopen(url) as response, open(test_cases_filename, 'wb') as out_file:
    out_file.write(response.read())

print('Downloaded test case archive to {}'.format(os.path.abspath(test_cases_filename)))

## Step 3: Extract the Test Case Archive

In [38]:
import os, errno, zipfile

# extract to 'archives' directory
with zipfile.ZipFile(test_cases_filename) as z:
    z.extractall('.')
    
print('Extracted test cases to {}'.format(os.path.abspath('.')))

Extracted test cases to /Users/phantom/devel/src/tellurium-combine-archive-test-cases/sbml-test-suite


## Step 4: Collect All Test Cases

Substitute a specific value here for cases if you only want to convert a subset, e.g. `cases = ['00001','00002','00003']`.

In [25]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

# Find all sbml test cases
root_path = os.path.abspath('cases/semantic/')
cases = [case for case in os.listdir(root_path) if is_number(case)]
# cases = sorted(cases) # all cases
cases = ['00001','00002','00003'] # subset

print('cases:')
print(cases)

cases:
['00001', '00002', '00003']


## Step 5: Create COMBINE Archives

Here, we create COMBINE archives for SBML Level 3 Version 1. We could package all levels and versions in the same COMBINE archive, but this would drastically increase simulation time.

First, we will set the level and version used to pick out only L3V1 SBML test cases (all SED-ML in this example is L1V1):

In [36]:
lv_string = 'l3v1'

Next, we create the directory structure for the archives

In [45]:
# create a function to make a new directory if it doesn't already exist
def mkdirp(path):
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

# make a new directory for this level and version if it doesn't already exist
lv_archive_path = os.path.join('archives', lv_string)
mkdirp(lv_archive_path)

print('Created directory {}'.format(lv_archive_path))

Created directory archives/l3v1


Next, we create the archives.

In [46]:
import re
from xml.dom import minidom
import xml.etree.ElementTree as ET

for case in cases:
    test_case_path = os.path.join(root_path, case)
    ls = os.listdir(test_case_path)
    # Only L3V1:
    regex_sbml = re.compile(case + '-sbml-{}\.xml'.format(lv_string), re.IGNORECASE)
    regex_sedml = re.compile(case + '-sbml-{}\-sedml.xml'.format(lv_string), re.IGNORECASE)
    # All levels/versions:
    # regex_sbml = re.compile(case + '-sbml-l\dv\d\.xml', re.IGNORECASE)
    # regex_sedml = re.compile(case + '-sbml-l\dv\d\-sedml.xml', re.IGNORECASE)

    sbmlfiles = sorted([file for file in ls if regex_sbml.search(file)])
    sedmlfiles = sorted([file for file in ls if regex_sedml.search(file)])
    plot_file = [file for file in ls if 'plot.jpg' in file][0]

    ET.register_namespace('', 'http://identifiers.org/combine.specifications/omex-manifest')
    manifest_template = '''<?xml version="1.0" encoding="UTF-8"?>
    <omexManifest
        xmlns="http://identifiers.org/combine.specifications/omex-manifest"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
        xsi:schemaLocation="http://identifiers.org/combine.specifications/omex-manifest combine.xsd "></omexManifest>
    '''

    doc = ET.fromstring(manifest_template)
    manifest = ET.SubElement(doc, 'content')
    manifest.attrib['format'] = 'http://identifiers.org/combine.specifications/omex-manifest'
    manifest.attrib['location'] = './manifest.xml'

    for sbmlfile in sbmlfiles:
        model = ET.SubElement(doc, 'content')
        model.attrib['format'] = 'http://identifiers.org/combine.specifications/sbml'
        model.attrib['location'] = './' + sbmlfile

    for sedmlfile in sedmlfiles:
        sedml = ET.SubElement(doc, 'content')
        sedml.attrib['format'] = 'http://identifiers.org/combine.specifications/sed-ml.level-1.version-1'
        sedml.attrib['location'] = './' + sedmlfile
        sedml.attrib['master'] = 'true'

    xml_str = ET.tostring(doc, encoding='UTF-8')
    # reparse the xml string to pretty print it
    reparsed = minidom.parseString(xml_str)
    pretty_xml_str = reparsed.toprettyxml(indent="    ")

    # use zipfile to create Combine archive containing
    from zipfile import ZipFile
    archive_name = case + '.omex'
    archive_path = os.path.join('archives', lv_string, archive_name)
    initial_wd = os.getcwd()
    ls = os.listdir(test_case_path)
    with ZipFile(archive_path, 'w') as archive:
        # write the manifest
        archive.writestr('manifest.xml', pretty_xml_str.encode('utf-8'))
        os.chdir(test_case_path)
        for f in sbmlfiles+sedmlfiles:
            archive.write(f)

    os.chdir(initial_wd)
    
    # print the number of contained files (add +1 for the manifest)
    print('Created {} (containing {} files)'.format(archive_path, 1+len(sbmlfiles+sedmlfiles)))


Created archives/l3v1/00001.omex (containing 3 files)
Created archives/l3v1/00002.omex (containing 3 files)
Created archives/l3v1/00003.omex (containing 3 files)


## Step 6: Run all simulations in the COMBINE archives in Tellurium.

In [51]:
import tellurium as te

for case in os.listdir(lv_archive_path):
    print('Running {}'.format(case))
    te.convertAndExecuteCombineArchive(os.path.join(lv_archive_path, case))